✅ **导入图片总结表格：**

|        方法         |    返回类型     | 是否支持多种格式 | 是否适合深度学习 |
|:-----------------:|:-----------:|:--------:|:--------:|
|  PIL.Image.open   |  PIL.Image  |    ✅     | ⚠️（需转换）  |
| skimage.io.imread | NumPy (RGB) |    ✅     |    ✅     |
|    cv2.imread     | NumPy (BGR) |    ✅     |    ✅     |
|    plt.imread     | NumPy (RGB) |  ⚠️（有限）  |    ⚠️    |
|  imageio.imread   | NumPy (RGB) |    ✅     |    ✅     |
|  tf.image.decode  |   Tensor    |    ✅     |    ✅     |


1️⃣ **PIL ↔ NumPy**
| 方向          | 代码                                                  | 备注                   |
| :-----------: | :---------------------------------------------------: | --------------------: |
| PIL → NumPy | `img_np = np.array(pil_img)`                        | RGB顺序                |
| NumPy → PIL | `pil_img = Image.fromarray(img_np.astype('uint8'))` | 必须是`uint8`，形状(H,W,C) |


2️⃣ **NumPy ↔ Tensor**
| 框架         | 方向             | 代码                                                       | 备注                              |
| ---------- | -------------- | -------------------------------------------------------- | ------------------------------- |
| PyTorch    | NumPy → Tensor | `tensor = torch.from_numpy(img_np).permute(2,0,1)`       | HWC→CHW，归一化到0-1再`/255.`         |
| PyTorch    | Tensor → NumPy | `img_np = tensor.mul(255).permute(1,2,0).byte().numpy()` | CHW→HWC                         |
| TensorFlow | NumPy → Tensor | `tensor = tf.convert_to_tensor(img_np)`                  | 可选`tf.cast(tensor, tf.float32)` |
| TensorFlow | Tensor → NumPy | `img_np = tensor.numpy()`                                | 必须在Eager模式下                     |

3️⃣ **PIL ↔ Tensor（跳过NumPy）**
| 框架         | 方向           | 代码                                                                | 备注             |
| ---------- | ------------ | ----------------------------------------------------------------- | -------------- |
| PyTorch    | PIL → Tensor | `tensor = transforms.ToTensor()(pil_img)`                         | 已归一化到0-1，CHW   |
| PyTorch    | Tensor → PIL | `pil_img = transforms.ToPILImage()(tensor)`                       | 自动反归一化，CHW→HWC |
| TensorFlow | PIL → Tensor | `tensor = tf.image.decode_image(tf.io.encode_jpeg(pil_to_bytes))` | 不推荐，建议先转NumPy  |
| TensorFlow | Tensor → PIL | `pil_img = Image.fromarray(tensor.numpy().astype('uint8'))`       | 先转NumPy        |

⚠️ **常见坑提醒**
| 坑点          | 说明                                                   |
| :-----------: | :----------------------------------------------------: |
| **通道顺序**    | OpenCV/PIL是HWC，PyTorch是CHW；TensorFlow默认是NHWC（含batch） |
| **数据类型**    | PIL和NumPy必须是`uint8`；Tensor是`float32`且值域0-1           |
| **归一化**     | `ToTensor()`已帮你`/255.`；自己转NumPy时记得手动除                |
| **batch维度** | 推理时PyTorch常用`(1,3,H,W)`，TensorFlow常用`(1,H,W,3)`      |
